In [2]:
import numpy as np
import json
import itertools
import re
import matplotlib.pyplot as plt
import matplotlib as mpl

## Code For Loading

In [32]:
def str2mat(s):
    rows = []
    N = len(s.split(','))
    env = {'x': np.array([1,0,0]), 'y': np.array([0,1,0]), 'z': np.array([0,0,1])}
    fake_env = {'x': 0, 'y': 0, 'z': 0}
    for i, si in enumerate(s.split(',')):
        # treat implicit multiplication - 2x = 2 * x
        si = re.sub('(?<=\d)(?=x) | (?<=\d)(?=y) | (?<=\d)(?=z)', '*', si, flags=re.X)
        r = [0] * N
        l = {}
        # use fake ones to get translation
        exec('translation = ' + si.strip(), fake_env, l)
        exec('scale = ' + si.strip(), env, l)
        # expand 
        if type(l['scale']) != np.ndarray:
            t = np.zeros(N)
            t[i] = l['translation']
            l['translation'] = t
            l['scale'] = t
        # remove trans and add
        rows.append(np.append((l['scale'] - l['translation'])[:N], np.sum(l['translation'])))
    rows.append(np.array(N * [0] + [1]))
    result = np.vstack(rows)    
    return result
def asymm_constraints(s):
    s = s.replace('≤', '<=')        
    env = {}
    in3d = 'z' in s
    exec('from math import *', env)
    funcs = []
    for i, si in enumerate(s.split(';')):
        # treat implicit multiplication - 2x = 2 * x
        si = re.sub('(?<=\d)(?=x) | (?<=\d)(?=y) | (?<=\d)(?=z)', '*', si, flags=re.X)
        l = {}
        if in3d:
            exec(f'l{i} = lambda x,y,z:' + si, env, l)
        else:
            exec(f'l{i} = lambda x,y:' + si, env, l)
        funcs.append(l[f'l{i}'])
    if in3d:
        return lambda x,y,z: sum([f(x,y,z) for f in funcs]) == len(funcs)
    else:
        return lambda x,y: sum([f(x,y) for f in funcs]) == len(funcs)
projectors2d = {             
    'Square': 
    np.array([
        4 * [1], 
        4 * [0],
        4 * [0],
        4 * [1]
    ]),
    'Rectangular': 
    np.array([
        [1, 1, 0, 0], 
        4 * [0],
        4 * [0],
        [0, 0, 1, 1]
    ]),
    'Hexagonal':
    np.array([
        4 * [1],
        4 * [-1/2],
        4 * [0],
        4 * [np.sqrt(3)/2]
    ]),
    'Oblique':np.eye(4),
}

projectors3d = {             
    'Hexagonal':
    np.array([
        3 * [1] + 6 * [0], #ax
        3 * [-1/2] + 6 * [0], #bx
        9 * [0], #cx
        9 * [0], #ay
        3 * [0] + 3 * [np.sqrt(3)/2] + 3 * [0], #by
        9 * [0], #cy
        9 * [0], #az
        9 * [0], #bz,
        6 * [0] + 3 * [1], #cz        
    ]),
    'Cubic':
        np.array([
        3 * [1] + 6 * [0], #ax
        9 * [0], #bx
         9 * [0], #cx
        9 * [0], #ay
        3 * [0] + 3 * [1] + 3 * [0], #by
        9 * [0], #cy
        9 * [0], #az
        9 * [0], #bz,
        6 * [0] + 3 * [1], #cz        
    ]),    
    'Tetragonal':
    np.array([
        6 * [1] + 3 * [0], #ax
        9 * [0], #bx
        9 * [0], #cx
        9 * [0], #ay
        6 * [1] + 3 * [0], #by
        9 * [0], #cy
        9 * [0], #az
        9 * [0], #bz,
        6 * [0] + 3 * [1], #cz
    ]),
    'Triclinic':np.eye(9),
    'Monoclinic': #TODO: might be missing potential rotation around z
        np.array([
        6 * [1] + 3 * [0], #ax
        9 * [0], #bx
        2 * [0] + [1] + 6 * [0], #cx
        9 * [0], #ay
        6 * [1] + 3 * [0], #by
        5 * [0] + [1] + 3 * [0], #cy
        9 * [0], #az
        9 * [0], #bz,
        8 * [0] + [1], #cz
    ]),
    'Orthorhombic': #TODO: might be missing potential rotation around z
        np.array([
        3 * [1] + 6 * [0], #ax
        9 * [0], #bx
        9 * [0], #cx
        9 * [0], #ay
        3 * [0] + 3 * [1] + 3 * [0], #by
        9 * [0], #cy
        9 * [0], #az
        9 * [0], #bz,
        6 * [0] +  3 * [1], #cz
    ]),
}
projectors3d['Trigonal'] = projectors3d['Hexagonal']

def write_group(f, name, group, dim):
    # tiling code needs to be updated for more than 2D
    def fmt(n):
        if n is None:
            return list(np.round(np.eye(dim**2).astype(float).flatten(), 8))
        return list(np.round(n.astype(float).flatten(), 8))
    try:
        projector = projectors2d[group['lattice']] if dim == 2 else projectors3d[group['lattice']]
    except KeyError:
        projector = None
    key = 'genpos'
    if key not in group:
        key = 'sites'
    members = [str2mat(s) for s in group[key]]
    result = {'name': name, 'size': len(members), 'members': [], 'projector': fmt(projector)}
    for m in members:
        result['members'].append(fmt(m))
    # should be same for all, so use last
    dof = np.sum(np.sum(m[:-1, :-1]**2, axis=1) > 0)
    result['dof'] = int(dof)
    json.dump(result, f, indent=True)
    print('Wrote group with', len(members), 'members')

In [35]:
def load_group(gnum, dim):
    gnum = str(gnum)
    with open(f'{dim}dgroups.json', 'r') as f:        
        all_groups = json.load(f)
    if gnum not in all_groups:
        raise KeyError('Could not find group ' + gnum)
    group = all_groups[gnum]
    return group
def prepare_input(gnum, dim, N, name):
    group = load_group(gnum, dim)
    asymm_unit = asymm_constraints(group['asymm_unit'])
    with open(f'{name}.json', 'w') as f:
        write_group(f, name, group, dim)
    for i, g in enumerate(group['specpos']):
        with open(f'{name}-{i:02d}.json', 'w') as f:
            write_group(f, name + f'-{i}', g, dim)
    Ni = N
    with open(f'{name}.xyz', 'w') as f:
        while Ni > 0:
            x = np.random.uniform()
            y = np.random.uniform()
            z = np.random.uniform()
            if (dim == 2 and asymm_unit(x,y)) or (dim == 3 and asymm_unit(x,y,z)):                
                f.write(f'{x} {y}\n' if dim == 2 else f'{x} {y} {z}\n')
                Ni -= 1

In [36]:
prepare_input(5, 2, 16, 'wp-5')

Wrote group with 4 members
Wrote group with 2 members


In [29]:
# check them all
for i in range(1, 18):
    prepare_input(i, 2, 16, 'test')
for i in range(1, 231):
    prepare_input(i, 3, 16, 'test')

Wrote group with 1 members
Wrote group with 2 members
Wrote group with 1 members
Wrote group with 1 members
Wrote group with 1 members
Wrote group with 1 members
Wrote group with 2 members
Wrote group with 1 members
Wrote group with 1 members
Wrote group with 2 members
Wrote group with 4 members
Wrote group with 2 members
Wrote group with 4 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 1 members
Wrote group with 1 members
Wrote group with 1 members
Wrote group with 1 members
Wrote group with 4 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 4 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 8 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 4 members
Wrote group with 2 members
Wrote group with 1 members
W

Wrote group with 2 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 8 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 8 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 8 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 8 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 8 members
W

Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 1 members
Wrote group with 1 members
Wrote group with 1 members
Wrote group with 1 members
Wrote group with 8 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 2 members
Wrote group with 8 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 8 members
Wrote group with 4 members
Wrote group with 8 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 4 members
Wrote group with 2 members
Wrote group with 2 members
W

KeyboardInterrupt: 

## Generating Groups

**Should not need to execute!!**

In [ ]:
import time
import requests
from bs4 import BeautifulSoup as BS

def get_html(group_no, dim=3):
    if dim == 3:
        url = f'https://it.iucr.org/Ac/ch2o3v0001/sgtable2o3o{group_no:03}/'
    elif dim == 2:
        url = f'https://it.iucr.org/Ac/ch2o2v0001/sgtable2o2o{group_no:03}/'
    result = requests.get(url)
    return result.content
def combine(s, t):
    '''
    Combine string of coords. Example:
    combine("x, y, z - y", "0, 0, 1/2")
    will return "x, y, z - y + 1/2"
    '''
    s = s.split(',')
    t = t.split(',')
    for i in range(len(s)):
        s[i] = s[i] + ' + ' + t[i]
    return ','.join(s)
def parse_trans_str(text):
    '''
    Split trans string into individual pieces
    '''
    text = text.replace('+', '')
    r = []
    for t in text.split('('):        
        r.append(t.replace(')', '')) 
    # filter out empties
    return [ri for ri in r if ri]
def parse_group(html):
    b = BS(html)
    gen_table = b.select_one('div > table.genpos')
    # sorry
    trans_str = []
    try:
        # find the first row after header
        index = 0
        for tr in gen_table.find_all('tr'):
            if 'Coordinates' in tr.text:
                continue
            if len(tr.text) < 5:
                continue
            break
        trans_str = ''.join(tr.find_all('td')[1].text.split())
        trans_str = parse_trans_str(trans_str)
    except IndexError:
        pass
    intable = False
    sites = []
    for t in gen_table.find_all('table'):
        for row in t.find_all('tr'):
            data = row.find_all('td')
            if intable:
                for d in data:                
                    if len(d.text.split(')')) == 1:
                        intable = False
                        break
                    text =  ''.join(d.text.split(')')[-1].split())     
                    if trans_str:
                        for tr in trans_str:
                            sites.append(combine(text, tr))
                    else:
                        sites.append(text)
            else:
                try:
                    multi = int(data[0].text)
                    letter = data[1].text
                    sym = data[2].text
                    sites = []
                    intable = True
                except:
                    pass
                    sites = []
    spec_table = b.select_one('div > table.specpos')
    specs = []
    if spec_table:
        # not sure why we cannot use tr.specpos
        intable = False
        wsites = None # will be set - can't figure out why we're getting repeats
        for w in spec_table.find_all('tr'):
            info_table = w.select_one('table')
            if info_table:
                # save old one
                if wsites is not None:
                    # convert to matrices
                    wsites = list(wsites)
                    assert len(wsites) == mpc, 'Only had sites ' + str(wsites) + ' but needed ' + str(mpc)
                    specs.append({'name': name, 'size': mpc, 'sites': wsites})                
                mpc, name, *_ = info_table.text.split()
                mpc = int(mpc)
                intable = True
                wsites = set()
            if intable:
                for td in w.find_all('td'):  
                    if 'class' in td.attrs and 'specposcoords' in td.attrs['class']:
                        s = ''.join(td.text.split())
                        if trans_str:
                            for tr in trans_str:
                                c = combine(s, tr)
                                wsites.add(c)
                        else:
                            wsites.add(s)
        if wsites is not None:
            # convert to matrices
            wsites = list(wsites)
            assert len(wsites) == mpc, 'Only had sites ' + str(wsites) + ' but needed ' + str(mpc)
            specs.append({'name': name, 'size': mpc, 'sites': wsites})    
    asymm = b.select('td.asymmetricunit')[1].text
    asymm = ''.join(asymm.split())
    p = b.select_one('tr.sgheader').select('td.sgheader')[-1].text.strip()
    data = {'lattice': p, 'genpos': sites, 'asymm_unit': asymm, 'specpos': specs}
    return data
def get_group(number, dim=3):
    return parse_group(get_html(number, dim=dim))

In [ ]:
all_groups = {}
for g in range(1,231):
    time.sleep(1)
    group = get_group(g, 3)
    all_groups[g] = group
with open('3dgroups.json', 'w') as f:
    json.dump(all_groups, f, indent=True)

In [ ]:
all_groups = {}
for g in range(1,18):
    time.sleep(1)
    group = get_group(g, 2)
    all_groups[g] = group
with open('2dgroups.json', 'w') as f:
    json.dump(all_groups, f, indent=True)

## Making Pictures

In [ ]:
def plot_group(x, basis, g, title, show_basis=False, colors='black', n_images=2):    
    basis = (g[0] @ basis.flatten()).reshape(basis.shape)    
    if show_basis:
        plt.plot([0, basis[0,0]], [0,basis[1,0]], '-', color='black')
        plt.plot([0, basis[0,1]], [0,basis[1,1]], '-', color='black')
    n = list(range(-n_images, n_images + 1))
    # reduce x size to match group
    npoints = x.shape[0] // len(g[1])
    x = x[:npoints, :]    
    points = []
    if type(colors) is str:
        colors = [colors] * x.shape[0]
    for i, ns in enumerate(itertools.product(n, repeat=2)):        
        for w in g[1]:
            xw = np.mod((w @ x.T).T, 1)
            xc = (xw[:,:2] + ns) @ basis.T 
            plt.scatter(xc[:,0], xc[:,1], c=colors[:xc.shape[0]], marker='.', alpha=0.4,  edgecolors='none' )
            points.append(xc[:,:2])
    x = np.vstack(points)
    #sns.kdeplot(x=x[:,0], y=x[:,1], shade=True, bw_adjust=0.2, cmap='Reds')
    if title:
        plt.title(title)
    plt.gca().set_aspect('equal')
    plt.axis('off')
    plt.gca().set_facecolor('#f5f4e9')
    plt.gcf().patch.set_facecolor('#f5f4e9')

def generate_points(x, basis, g):
    basis = (g[0] @ basis.flatten()).reshape(basis.shape)    
    n = [0, -1, 1, -2, 2]
    # reduce x size to match group
    npoints = x.shape[0] // len(g[1])
    x = x[:npoints, :]   
    points = []
    for i, ns in enumerate(itertools.product(n, repeat=2)):        
        for w in g[1]:
            xw = np.mod((w @ x.T).T, 1)
            xc = (xw[:,:2] + ns) @ basis
            points.append(xc[:,:2])
    x = np.vstack(points)
    return x
np.random.seed(0)
npoints = 96
x = np.hstack((np.random.uniform(size=(npoints,2)) - 0.5, np.ones((npoints,1))))
basis = np.array([[1,0], [0,1]]) + np.random.uniform(size=(2,2))
base_colors = ["f94144","f3722c","f8961e","f9844a","f9c74f","90be6d","43aa8b","4d908e","577590","277da1"]    
colors = ['#' + base_colors[i % len(base_colors)] for i in range(x.shape[0])]
for i in range(1, 18):
    group = load_group(i, 2)
    plt.figure(figsize=(8,8))
    projector = projectors2d[group['lattice']]
    members = [str2mat(i) for i in group['genpos']]
    plot_group(x, basis, (projector, members), title=None, show_basis=True, colors='black')
    for j,sp in enumerate(group['specpos']):
        members = [str2mat(i) for i in sp['sites']]
        plot_group(x, basis, (projector, members), title=None,
                   show_basis=False, colors=['#' + base_colors[j]] * x.shape[0])
    plt.tight_layout()
    plt.savefig(f'2d_{i}' + '.png', dpi=90)

In [ ]:
# make a movie
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage

data = []
titles = []
for k,v in wallpaper.items():
    data.append(generate_points(x, basis, v))
    titles.append(k)
    
# make it loop
data.append(data[0])
titles.append(titles[0])
def CubicEaseInOut(p):
    if (p < 0.5):
        return 4 * p * p * p
    else:
        f = ((2 * p) - 2)
        return 0.5 * f * f * f + 1

scale = 2
duration = (len(data) - 1) * scale
dpi = 90
fig, ax = plt.subplots(figsize=(800 / dpi, 800 / dpi), dpi=dpi)
points = ax.plot(data[0][:,0], data[1][:,1], color='#333333', marker='.', markeredgewidth=0.0, linestyle='None', alpha=0.2)[0]
ax.set_facecolor('#f5f4e9')
fig.patch.set_facecolor('#f5f4e9')
title = ax.set_title('', fontsize=32, color='#333333',fontname='monospace')
ax.set_aspect(1.0/ax.get_data_ratio(), adjustable='box')
ax.axis('off')
#ax.set_xlim(-3,3)
#ax.set_ylim(-3,3)
#plt.tight_layout()
def make_frame(t):
    t /= scale
    s = t % 1
    if s < 0.25:
        s = CubicEaseInOut(s * 4)
    else:
        s = 1
    i = int(t) + 1
    p = data[i] * s + (1 - s) * data[i - 1]
    title.set_text(titles[i])
    points.set_data(p[:,0], p[:,1])
    ax.set_facecolor('#f5f4e9')
    return mplfig_to_npimage(fig)

animation = VideoClip(make_frame, duration=duration)
animation.write_gif('matplotlib.gif', fps=15)

